In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score

In [3]:
#read the data
final_vote=pd.read_csv('clean_vote_vect.csv')

In [4]:
final_vote.head()

,text,vote,help,recommend,helpful,funny
0,refin gameplay layer mechan top tradit clicker...,Recommended,14 people found this review helpful,1,14.0,NaN
1,complaint review game truli idl want write rev...,Recommended,9 people found this review helpful,1,9.0,NaN
2,clicker game time manag idl game pro great sou...,Recommended,7 people found this review helpful,1,7.0,NaN
3,play game year kept consist interest time freq...,Recommended,26 people found this review helpful4 people fo...,1,26.0,4.0
4,play idl clicker increment game year definit s...,Recommended,10 people found this review helpful,1,10.0,NaN


In [5]:
#split training and testing
X_train,X_test,y_train,y_test=train_test_split(final_vote['text'],
                                              final_vote['recommend'],
                                              test_size=0.25,
                                              random_state=0)

In [6]:
#set type
X_train=X_train.values.astype('U')
X_test=X_test.values.astype('U')

## 1-3 gram

In [7]:
#1-gram
vect=CountVectorizer(ngram_range=(1,1),min_df=5).fit(X_train)
vect_name=vect.get_feature_names()
X_train_vectorized = vect.transform(X_train)
full_text_matrix=vect.transform(final_vote['text'].values.astype('U'))

In [8]:
# 2-gram
vect_bigram=CountVectorizer(ngram_range=(1,2),min_df=5).fit(X_train)
X_train_bivectorized = vect_bigram.transform(X_train)

In [9]:
# 3-gram
vect_trigram=CountVectorizer(ngram_range=(1,3),min_df=5).fit(X_train)
X_train_trivectorized = vect_trigram.transform(X_train)

In [ ]:
#sparce matrix
text_vector=pd.DataFrame(full_text_matrix.toarray(),columns=vect.get_feature_names())
text_vector.head()

,aa,aaa,aaaand,aaand,aand,aard,ab,aback,abandon,abandonwar,...,zombi,zombifi,zomboid,zone,zoo,zoom,ztd,zu,zula,zweihand
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## logistic regression

In [1]:
#logistic regression
from sklearn.linear_model import LogisticRegression
# 1-gram
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
score_pred = model.predict(vect.transform(X_test))
# 2-gram
model_bigram=LogisticRegression()
model_bigram.fit(X_train_bivectorized,y_train)
predictions_bigram=model_bigram.predict(vect_bigram.transform(X_test))

NameError: name 'X_train_vectorized' is not defined

In [ ]:
# 3-gram
model_trigram=LogisticRegression()
model_trigram.fit(X_train_trivectorized,y_train)
predictions_trigram=model_trigram.predict(vect_trigram.transform(X_test))

In [22]:
#results for logstic regression
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score
# 1-gram
print(confusion_matrix(score_pred,y_test))
print(f1_score(score_pred,y_test))
print(accuracy_score(score_pred,y_test))
# 2-gram
print(confusion_matrix(predictions_bigram,y_test))
print(f1_score(predictions_bigram,y_test))
print(accuracy_score(predictions_bigram,y_test))
# 3-gram
print(confusion_matrix(predictions_trigram,y_test))
print(f1_score(predictions_trigram,y_test))
print(accuracy_score(predictions_trigram,y_test))

[[ 3852   825]
 [ 2262 18529]]
0.9231037489102005
0.8787890686351499
[[ 4347   826]
 [ 1767 18528]]
0.9346011248707408
0.8981859588503219
[[ 4338   838]
 [ 1776 18516]]
0.9340664884225395
0.8973613946913774


## random forest

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators': range(10,100,10), 'max_depth': range(1,10)}

gscv=GridSearchCV(RandomForestClassifier(),param_grid,scoring= 'accuracy',cv=10)
gscv.fit(X_train_vectorized, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [24]:
print("The optimal parameters: \nmax_depth:{} n_estimators:{}".
      format(gscv.best_params_["max_depth"],gscv.best_params_["n_estimators"]))
print(gscv.best_score_)

The optimal parameters: 
max_depth:9 n_estimators:10
0.7584518932502651


In [25]:
# 1-gram
rf1_model = RandomForestClassifier(max_depth=gscv.best_params_["max_depth"],
                                   n_estimators=gscv.best_params_["n_estimators"])
rf1_model.fit(X_train_vectorized, y_train)
rf1_predict=rf1_model.predict(vect.transform(X_test))

In [27]:
print(confusion_matrix(rf1_predict,y_test))
print(f1_score(rf1_predict,y_test))
print(accuracy_score(rf1_predict,y_test))

[[   68     8]
 [ 6046 19346]]
0.8647029902114156
0.7622899324642689


In [29]:
# 2-gram
rf2_model = RandomForestClassifier(max_depth=gscv.best_params_["max_depth"],
                                   n_estimators=gscv.best_params_["n_estimators"])
rf2_model.fit(X_train_bivectorized, y_train)
rf2_predict=rf2_model.predict(vect_bigram.transform(X_test))

In [30]:
print(confusion_matrix(rf2_predict,y_test))
print(f1_score(rf2_predict,y_test))
print(accuracy_score(rf2_predict,y_test))

[[    4     1]
 [ 6110 19353]]
0.8636454916661087
0.7600518297471337


In [31]:
# 3-gram
rf3_model = RandomForestClassifier(max_depth=gscv.best_params_["max_depth"],
                                   n_estimators=gscv.best_params_["n_estimators"])
rf3_model.fit(X_train_trivectorized, y_train)
rf3_predict=rf3_model.predict(vect_trigram.transform(X_test))

In [32]:
print(confusion_matrix(rf3_predict,y_test))
print(f1_score(rf3_predict,y_test))
print(accuracy_score(rf3_predict,y_test))

[[    3     1]
 [ 6111 19353]]
0.8636262216073898
0.7600125647871839


## svm

In [11]:
from sklearn import svm

In [10]:
# 1-gram
svm1 = svm.SVC(gamma='scale')
svm1.fit(X_train_vectorized, y_train)
svm1_predict=svm1.predict(vect.transform(X_test))

In [13]:
print(confusion_matrix(svm1_predict,y_test))
print(f1_score(svm1_predict,y_test))
print(accuracy_score(svm1_predict,y_test))

[[ 3142   489]
 [ 2972 18865]]
0.9159767910465879
0.8641039736139469


In [12]:
# 2-gram
svm2 = svm.SVC(gamma='scale')
svm2.fit(X_train_bivectorized, y_train)
svm2_predict=svm2.predict(vect_bigram.transform(X_test))

In [13]:
print(confusion_matrix(svm2_predict,y_test))
print(f1_score(svm2_predict,y_test))
print(accuracy_score(svm2_predict,y_test))

[[ 3172   447]
 [ 2942 18907]]
0.9177487076183772
0.8669310507303283


In [15]:
# 3-gram
svm3 = svm.SVC(gamma='scale')
svm3.fit(X_train_trivectorized, y_train)
svm3_predict=svm3.predict(vect_trigram.transform(X_test))

In [16]:
print(confusion_matrix(svm3_predict,y_test))
print(f1_score(svm3_predict,y_test))
print(accuracy_score(svm3_predict,y_test))

[[ 3172   448]
 [ 2942 18906]]
0.9177224406582204
0.8668917857703785


In [ ]:
## Naive Bayes